In [ ]:
!pip install scikit-learn==0.23.2

In [ ]:
!pip install keract

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random
import cv2
import os

import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report ,confusion_matrix
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.preprocessing import image
import pickle
from tensorflow.keras.applications import densenet
from glob import glob
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline

import class_activation_map as cam
import preprocessing 

In [ ]:
base_path = '../input/cell-images-for-detecting-malaria/cell_images'

In [ ]:
Dataset = preprocessing.load_and_preprocess_images(base_path)

In [ ]:
Data = pd.DataFrame(Dataset, columns = ['Images', 'Labels'])

In [ ]:
Data.head()

In [ ]:
def Labels_to_numeric(x):
        if x == 'Parasitized':
          return 1
        if x == 'Uninfected':
          return 0
Data['Labels'] = Data['Labels'].apply(Labels_to_numeric)
print(Data.head())

In [ ]:
x = np.array([i for i in Data['Images']]).reshape(-1, 64, 64, 3)
y = to_categorical([i for i in Data['Labels']])

In [ ]:
len(x)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, shuffle = True, random_state = 2)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

In [ ]:
X_train = X_train/255.
X_val = X_val/255.

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, FalseNegatives, FalsePositives, TruePositives, TrueNegatives


In [ ]:
# defining evaluation metrics

precision = Precision()
recall = Recall()
false_positive = FalsePositives()
false_negative = FalseNegatives()
true_positive = TruePositives()
true_negative = TrueNegatives()

In [ ]:
class myCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs = {}):
        if (logs.get('accuracy') > 0.98):
            self.model.stop_training = True

myCallback = myCallback()

In [ ]:
adam = Adam(learning_rate = 0.001)
# sgd = SGD(learning_rate = 0.0001, momentum = 0.8, nesterov = True)

### DenseNet

In [ ]:
model_d = densenet.DenseNet121(weights="imagenet", 
                    include_top=False,
                    input_tensor=Input(shape = (64,64,3))
                    )

In [ ]:
z = model_d.output
z = GlobalAveragePooling2D()(z)
z = BatchNormalization()(z)
z = Dropout(0.5)(z)
z = Dense(1024,activation='relu')(z) 
z = Dense(512,activation='relu')(z) 
z = BatchNormalization()(z)
z = Dropout(0.5)(z)
preds=Dense(2,activation='softmax')(z)

In [ ]:
densenet_model = Model(inputs=model_d.input,outputs=preds)
densenet_model.summary()

In [ ]:
for layer in model_d.layers:
  layer.trainable = False

In [ ]:
densenet_model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy', precision, recall, true_positive, true_negative, false_positive, false_negative])

In [ ]:
densenet_history = densenet_model.fit(x = X_train, y = Y_train, validation_data = (X_val, Y_val), epochs = 30, callbacks = myCallback)

In [ ]:
#save the model in .h5 format
densenet_model.save(filepath = 'densenet_model', save_format = 'h5', include_optimizer = True)
densenet_model.save(filepath = 'densenet_model_tf', save_format = 'tf', include_optimizer = True)

In [ ]:
# plot the loss
plt.plot(densenet_history.history['loss'], label='train loss')
plt.plot(densenet_history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss.jpg')

# plot the accuracy
plt.plot(densenet_history.history['accuracy'], label='train acc')
plt.plot(densenet_history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
# plt.savefig('AccVal_acc.jpg')


# plot the Precision
plt.plot(densenet_history.history['precision'], label='train precision')
plt.plot(densenet_history.history['val_precision'], label='val precision')
plt.legend()
plt.show()
# plt.savefig('Precision.jpg')

# plot the Recall
plt.plot(densenet_history.history['recall'], label='train recall')
plt.plot(densenet_history.history['val_recall'], label='val recall')
plt.legend()
plt.show()
# plt.savefig('Recall.jpg')

In [ ]:
hist_df = pd.DataFrame(densenet_history.history)
hist_json_file = './densenet_history.json'
with open(hist_json_file, mode = 'w') as f:
  hist_df.to_json(f)

In [ ]:
def load_and_preprocess_test_images(img_path, img_size):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, img_size, cv2.INTER_CUBIC)

  img_copy = img.copy()
  img_copy = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
  img_copy = cv2.GaussianBlur(img_copy, (7,7), 0)
  edges = cv2.Canny(img_copy, threshold1 = 80, threshold2 = 160)
  edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
  final_img = cv2.addWeighted(img, 0.5, edges, 0.5, 0)
  final_img = np.expand_dims(final_img, axis = 0)
  final_img = final_img/255.
  return final_img

In [ ]:
# load model

model = load_model("./densenet_model")

In [ ]:
# load image

img = load_and_preprocess_test_images(img_path = "../input/cell-images-for-detecting-malaria/cell_images/Uninfected/C100P61ThinF_IMG_20150918_144823_cell_139.png", img_size = (64, 64))

In [ ]:
# run predictions
import time
start = time.time()
preds = model.predict(img)
end = time.time()
print('Time taken for inference : {}seconds'.format(round((end - start), 2)))

In [ ]:
preds

In [ ]:
# obtain class labels

class_indices = np.argmax(preds, axis = 1)

In [ ]:
if class_indices == 0:
  class_label = 'Uninfected'

else:
  class_label = 'Parasitized'

class_label

### Class Activation Mapping

In [ ]:
model_path = './densenet_model'
img_path = X_train[0]
cam.class_activation_map(img_path,model_path)